In [30]:
# blanket import modules
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
import pprint
from scipy.stats import linregress
import requests
import gmaps
import gmaps.geojson_geometries
from api_key import g_key
from matplotlib.colors import to_hex
from matplotlib.cm import inferno
#api key
gmaps.configure(api_key=g_key)

In [31]:
happy_cs = "Output/happy_comb-data.csv"

In [32]:
happy_df = pd.read_csv(happy_cs)
#trying to use this at the end 
happy_country=happy_df['Country']
happy_df

,RANK,Country,Happiness score,Explained by: Healthy life expectancy,Sunshine hours per Year,Average Income,Alcohol Consumption (L/year)
0,1,Finland,7.821,0.775,1858.00,18742.0,10.65
1,2,Denmark,7.636,0.777,1932.00,20304.0,10.13
2,3,Iceland,7.557,0.803,1326.00,19300.0,9.21
3,4,Switzerland,7.512,0.822,1932.75,25787.0,11.23
4,5,Netherlands,7.415,0.787,1662.00,19690.0,9.67
...,...,...,...,...,...,...,...
93,136,India,3.777,0.471,2517.02,1314.0,5.61
94,137,Zambia,3.760,0.306,2965.47,1045.0,4.46
95,138,Malawi,3.750,0.388,2795.87,733.0,4.08
96,139,Tanzania,3.702,0.425,3032.85,992.0,12.04


In [33]:
#making initial map with lines around country to fill later
countries_geojson = gmaps.geojson_geometries.load_geometry('countries')
fig = gmaps.figure()
happy_layer=gmaps.geojson_layer(countries_geojson)
# fig.add_layer(happy_layer)
# fig


In [34]:
# need to get the min and the max of happiness
# happy_df=happy_df.set_index(["Country"])

In [35]:
happy_df.dtypes

RANK                                       int64
Country                                   object
Happiness score                          float64
Explained by: Healthy life expectancy    float64
Sunshine hours per Year                  float64
Average Income                           float64
Alcohol Consumption (L/year)             float64
dtype: object

In [36]:
happy_col=happy_df["Happiness score"]
h_range=[]

for x in happy_col:
    happ_conv= x / 10
    h_range.append(happ_conv)
    
# happy_range=happ_max-happ_min

happy_df['Happiness Range'] = h_range
happy_df
happy_df["Country"].values

array(['Finland', 'Denmark', 'Iceland', 'Switzerland', 'Netherlands',
       'Sweden', 'Norway', 'Israel', 'Austria', 'Australia', 'Ireland',
       'Germany', 'Canada', 'United States', 'United Kingdom', 'Belgium',
       'France', 'Slovenia', 'Romania', 'Spain', 'Uruguay', 'Italy',
       'Malta', 'Lithuania', 'Slovakia', 'Estonia', 'Panama', 'Brazil',
       'Kazakhstan', 'Cyprus', 'Latvia', 'Serbia', 'Chile', 'Nicaragua',
       'Mexico', 'Croatia', 'Poland', 'El Salvador', 'Hungary',
       'Uzbekistan', 'Japan', 'Honduras', 'Portugal', 'Argentina',
       'Greece', 'South Korea', 'Philippines', 'Thailand', 'Moldova',
       'Colombia', 'Bosnia and Herzegovina', 'Mongolia', 'Bolivia',
       'China', 'Paraguay', 'Peru', 'Montenegro', 'Ecuador', 'Vietnam',
       'Russia', 'Armenia', 'Bulgaria', 'Ivory Coast', 'North Macedonia',
       'Albania', 'South Africa', 'Bangladesh', 'Algeria', 'Ukraine',
       'Morocco', 'Mozambique', 'Cameroon', 'Senegal', 'Georgia', 'Gabon',
       'Ir

In [40]:
# We will need to scale the Happiness values to lie between 0 and 1, this sets up for color ranges below
happ_max1=happy_df["Happiness Range"].max()
happ_min1=happy_df["Happiness Range"].min()
hap_range=happ_max1-happ_min1

def calculate_color(h_range):
    
#     Convert the GINI coefficient to a color

    # make gini a number between 0 and 1
    normalized_range = (h_range - happ_min1) / hap_range

    # invert gini so that high inequality gives dark color
    inverse_hap = 1.0 - normalized_range

    # transform the gini coefficient to a matplotlib color
    mpl_color = inferno(inverse_hap)

    # transform from a matplotlib color to a valid CSS color
    gmaps_color = to_hex(mpl_color, keep_alpha=False)

    return gmaps_color


In [41]:
colors = []
for feature in countries_geojson['features']:
    country_name = feature['properties']['name']
    print(country_name)
    try:
        h_range = happy_df.loc[happy_df["Country"]==country_name]["Happiness Range"].values
        print(h_range)
        color = calculate_color(h_range)
        print(color)
    except KeyError:
        # no happyy score for that country: return default color
        color = (0, 0, 0, 0.3)
        print(country_name+" not available")
    except ValueError: 
        color = (0, 0, 0, 0.3)
        
    print(color)    
    colors.append(color)
colors

Afghanistan
[]
(0, 0, 0, 0.3)
Angola
[]
(0, 0, 0, 0.3)
Albania
[0.5199]
#cb4149
#cb4149
Aland
[]
(0, 0, 0, 0.3)
Andorra
[]
(0, 0, 0, 0.3)
United Arab Emirates
[]
(0, 0, 0, 0.3)
Argentina
[0.5967]
#8d2369
#8d2369
Armenia
[0.5399]
#bc3754
#bc3754
American Samoa
[]
(0, 0, 0, 0.3)
Antarctica
[]
(0, 0, 0, 0.3)
French Southern and Antarctic Lands
[]
(0, 0, 0, 0.3)
Antigua and Barbuda
[]
(0, 0, 0, 0.3)
Australia
[0.7162]
#240c4f
#240c4f
Austria
[0.7163]
#240c4f
#240c4f
Azerbaijan
[]
(0, 0, 0, 0.3)
Burundi
[]
(0, 0, 0, 0.3)
Belgium
[0.6805]
#450a69
#450a69
Benin
[0.4623]
#ec6726
#ec6726
Burkina Faso
[0.467]
#eb6429
#eb6429
Bangladesh
[0.5155]
#ce4347
#ce4347
Bulgaria
[0.5371]
#bd3853
#bd3853
Bahrain
[]
(0, 0, 0, 0.3)
The Bahamas
[]
(0, 0, 0, 0.3)
Bosnia and Herzegovina
[0.5768]
#9d2964
#9d2964
Belarus
[]
(0, 0, 0, 0.3)
Belize
[]
(0, 0, 0, 0.3)
Bolivia
[0.56]
#ab2f5e
#ab2f5e
Brazil
[0.6293]
#721a6e
#721a6e
Barbados
[]
(0, 0, 0, 0.3)
Brunei
[]
(0, 0, 0, 0.3)
Bhutan
[]
(0, 0, 0, 0.3)
Botswana
[]


[(0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 '#cb4149',
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 '#8d2369',
 '#bc3754',
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 '#240c4f',
 '#240c4f',
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 '#450a69',
 '#ec6726',
 '#eb6429',
 '#ce4347',
 '#bd3853',
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 '#9d2964',
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 '#ab2f5e',
 '#721a6e',
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 '#320a5e',
 '#0b0724',
 '#7c1d6d',
 '#ad305d',
 '#c83f4b',
 '#d54a41',
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 '#9d2964',
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 '#771c6d',
 (0, 0, 0, 0.3),
 '#310a5c',
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 '#040314',
 (0, 0, 0, 0.3),
 '#d04545',
 '#b1325a',
 '#f8850f',
 (0, 0, 0, 0.3),
 '#62146e',
 '#6d186e',
 '#f8890c',
 '#000004',
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 '#510e6c',

In [42]:

fig = gmaps.figure()
happy_layer2 = gmaps.geojson_layer(
    countries_geojson,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=0.8)
fig.add_layer(happy_layer2)
fig

Figure(layout=FigureLayout(height='420px'))